In [28]:
code = '005930'  # Samsung elec.

In [29]:
import requests #http 요청을 보내는 모듈
url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
res = requests.get(url) #이 url로 get요청보내고, 응답 return값을 res변수에 받는다.
res.encoding = 'utf-8'

In [30]:
from bs4 import BeautifulSoup
soap = BeautifulSoup(res.text, 'lxml')

In [31]:
#page_navigation 마지막 번호(pg_last)
el_table_navi = soap.find("table", class_="Nnavi")
el_td_last = el_table_navi.find("td", class_="pgRR")
pg_last = el_td_last.a.get('href').rsplit('&')[1]
pg_last = pg_last.split('=')[1]
pg_last = int(pg_last)
pg_last

595

In [32]:
import traceback
import pandas as pd

def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0] #리스트에 [0]붙여서 dataframe 만들기
        _df = _df.dropna() #NaN값 제거
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [33]:
import datetime
str_datefrom = datetime.datetime.strftime(datetime.datetime(year=2018, month=1, day=1), '%Y.%m.%d')
str_dateto = datetime.datetime.strftime(datetime.datetime.today(), '%Y.%m.%d')


In [34]:
df = None
for page in range(1, pg_last+1):
    _df = parse_page(code, page)
    _df_filtered = _df[_df['날짜'] > str_datefrom]
    
    if df is None:
        df = _df_filtered
    else:
        df = pd.concat([df, _df_filtered])
    if len(_df) > len(_df_filtered):
        break

In [36]:
import os
path_dir = 'data/stockprice'
if not os.path.exists(path_dir):
    os.makedirs(path_dir)
path = os.path.join(path_dir, '{code}_{date_from}_{date_to}.csv'.format(code=code, date_from=str_datefrom, date_to=str_dateto))
path

'data/stockprice\\005930_2018.01.01_2020.02.14.csv'

In [37]:
df.to_csv(path, index=False,encoding='cp949')